Implementation Plan:

You can run git commands like this to get the latest version of code to work with:

git checkout preprod && git pull && git checkout -B uc-code-refactor-20230225 && git checkout uc-code-refactor-20230224 ./uc1Codefix.ipynb

In [0]:
import os, re, fnmatch

Function to find a given string and replace all occurances in all the notebooks (.py) in the current direcotry and its sub directories

In [0]:
def find_replace(directory, filename_filter, find, replace, filePattern="*.py") -> None:
    """traverses through the given directory and using given conditions and pattern, does a find + replace"""
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        print(path)
        for filename in fnmatch.filter(files, filePattern):
            if filename_filter.lower() in filename.lower():
                filepath = os.path.join(path, filename)
                with open(filepath) as f:
                    s = f.read()
                if find.lower() in s.lower():
                    s = s.replace(find, replace)
                    print (f"{filename} updated - [{find}] --> [{replace}]")
                with open(filepath, "w") as f:
                    f.write(s)


**MDP Dimension & Facts Notebooks Refactoring**

Some view notebooks have references that deviate from the standard naming convention, hence handling manually in the following command

In [0]:
directory   = "../Transform/Bridge/"

In [0]:
find_replace(directory, '', '# MAGIC %run ../../Common/common-transform', "# MAGIC %run ../../Common/common-transform \n\n# COMMAND ---------- \n\n# MAGIC %run ../../Common/common-helpers \n# COMMAND ---------- \n")

adding f string where get_table_name function was called in place of hardcoded table names

In [0]:
find_replace(directory, '', 'spark.sql("""', 'spark.sql(f"""')

In [0]:

findstr     = 'GetTable(f"{TARGET}.factcustomerinteraction")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{TARGET}\', \'factcustomerinteraction\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = 'GetTable(f"{SOURCE}.crm_skwg_brel")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{SOURCE}\', \'crm_skwg_brel\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = 'GetTable(f"{SOURCE}.crm_crmorderphio")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{SOURCE}\', \'crm_crmorderphio\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:

findstr     = 'GetTable(f"{TARGET}.dimcustomerserviceattachmentinfo")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{TARGET}\', \'dimcustomerserviceattachmentinfo\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:

findstr     = 'GetTable(f"{SOURCE}.crm_crmd_brelvonae")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{SOURCE}\', \'crm_crmd_brelvonae\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = 'GetTable(f"{TARGET}.dimcustomerserviceemailheader")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{TARGET}\', \'dimcustomerserviceemailheader\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = 'GetTable(f"{TARGET}.factcustomerservicerequest")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{TARGET}\', \'factcustomerservicerequest\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = 'GetTable(f"{TARGET}.factcustomerserviceworknote")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{TARGET}\', \'factcustomerserviceworknote\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:

findstr     = 'GetTable(f"{TARGET}.factsurveymiscellaneousinformation")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{TARGET}\', \'factsurveymiscellaneousinformation\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:

findstr     = 'GetTableName(f"{SOURCE}.maximo_relatedrecord")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{SOURCE}\', \'maximo_relatedrecord\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = 'GetTableName(f"{DEFAULT_TARGET}.factWorkOrder")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{DEFAULT_TARGET}\', \'factWorkOrder\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = 'GetTableName(f"{DEFAULT_TARGET}.factserviceTicket")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{DEFAULT_TARGET}\', \'factserviceTicket\')}")'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = '{TARGET}.factWorkOrder'
replacestr  = '{get_table_namespace(f\'{TARGET}\', \'factWorkOrder\')}'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = '{TARGET}.factpreventivemaintenance'
replacestr  = '{get_table_namespace(f\'{TARGET}\', \'factpreventivemaintenance\')}'
find_replace(directory, '', findstr, replacestr)

In [0]:
findstr     = 'hive_metastore.'
replacestr  = ''
find_replace(directory, '', findstr, replacestr)

Function to get the list of hardcoded cleansed tablenames in the view definitions, uses regex to pattern match 'cleansed.'

In [0]:
def get_strings_to_replace(full_path, pattern):
    strings_to_replace = []
    for filename in os.listdir(full_path):
        if filename.endswith(".py"):
            with open(os.path.join(full_path, filename), "r") as f:
                for line in f:
                    for word in line.split():
                        if pattern.search(word): 
                            if (len(word.split("."))) == 2:
                                strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                            else:
                                print (f"This was formatted poorly and is hence not included in the list:\n \'{filename} \'- {word}\n")
    return strings_to_replace

Finding and replacing the hardcoded cleansed table names in view definitions using the above function

In [0]:
directory = "../Transform/Bridge/"
patternstr = re.compile(r"\bcleansed.\b")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)

In [0]:
strings_to_replace

In [0]:

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, '', findstr, replacestr)

In [0]:
directory = "../Transform/Bridge/"
patternstr = re.compile(r"\braw.\b")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)


In [0]:
strings_to_replace

In [0]:

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, '', findstr, replacestr)

Replacing "curated.view\<name\>" with "\<env_\>curated.\<productdatabase\>.\<viewname\>" 
in notebooks under "/Databricks/MDP-Framework/Transform/Views/". These views are meant to be created in product specific database, resolved based on the csv lookup.

In [0]:
directory = "../Transform/Bridge/"
patternstr = re.compile(r"\bcurated.\b")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)


In [0]:
strings_to_replace

In [0]:

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    # schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'curated\', \'{table}\')}}'
    print(replacestr)
    find_replace(directory, '', findstr, replacestr)

In [0]:
def check_strings(full_path, pattern):
    for filename in os.listdir(full_path):
        if filename.endswith(".py"):
            with open(os.path.join(full_path, filename), "r") as f:
                for line in f:
                    for word in line.split():
                        if pattern.search(word): 
                            print (f"{word}\n found in {filename}")
                            # strings_to_replace.append(word)

    # return strings_to_replace

In [0]:
check_path = "../Transform/"
patternstr = re.compile(r'{DEFAULT_TARGET}')

strings_to_replace =  check_strings(full_path = directory, pattern = patternstr)
strings_to_replace